# Rewriting example from openMDAO
https://openmdao.org/newdocs/versions/latest/examples/beam_optimization_example.html

In [1]:
import numpy as np

In [2]:
from compute import Var
from datastructures.execution import Component

In [3]:
import numpy as np

In [21]:
T = np.random.random((10,4,4))

In [5]:
E = 1.
L = 1.
b = 0.1
volume = 0.01
num_elements = 10

In [6]:
fx = lambda b,h: 1./12.*b*h**3
Icomp = Component(fx, inputs=('b','h'), outputs=('I',), indims=(1,num_elements), outdims=(num_elements,))

In [7]:
fx = lambda b,h,L: b*sum(h)*L
Vcomp = Component(fx, inputs=('b','h','L'), outputs=('V',), indims=(1,num_elements,1), outdims=(1,))

In [42]:
Ival = Icomp.evaldict({'b':12, 'h':np.linspace(1, 1, num_elements)})

In [43]:
Ival

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [44]:
def compute_K_local(I):
    L0 = L / num_elements
    coeffs = np.empty((4, 4))
    coeffs[0, :] = [12, 6 * L0, -12, 6 * L0]
    coeffs[1, :] = [6 * L0, 4 * L0 ** 2, -6 * L0, 2 * L0 ** 2]
    coeffs[2, :] = [-12, -6 * L0, 12, -6 * L0]
    coeffs[3, :] = [6 * L0, 2 * L0 ** 2, -6 * L0, 4 * L0 ** 2]
    coeffs *= E / L0 ** 3
    print(coeffs)
    return np.concatenate([(coeffs * i).flat for i in Ival])

In [45]:
Kcomp = Component(compute_K_local, inputs=('I',), outputs=('K_local',), indims=(10,), outdims=(num_elements*16,))

In [46]:
K_local_flat = Kcomp.evaldict({'I':Ival})
K_local = K_local_flat.reshape(num_elements, 4, 4)

[[ 12000.    600. -12000.    600.]
 [   600.     40.   -600.     20.]
 [-12000.   -600.  12000.   -600.]
 [   600.     20.   -600.     40.]]


In [48]:
#def compute_K_augmented(K_local)
#K_local = K_local.reshape(num_elements, 4, 4)
num_nodes = num_elements + 1
num_entry = num_elements * 12 + 4
ndim = num_entry + 4

data = np.zeros((ndim, ))
cols = np.empty((ndim, ))
rows = np.empty((ndim, ))

# First element.
data[:16] = K_local[0,:,:].flat
cols[:16] = np.tile(np.arange(4), 4)
rows[:16] = np.repeat(np.arange(4), 4)

j = 16
for ind in range(1, num_elements):
    ind1 = 2 * ind
    K = K_local[ind, :, :]

    # NW quadrant gets summed with previous connected element.
    data[j-6:j-4] += K[0, :2]
    data[j-2:j] += K[1, :2]

    # NE quadrant
    data[j:j+4] = K[:2, 2:].flat
    rows[j:j+4] = np.array([ind1, ind1, ind1 + 1, ind1 + 1])
    cols[j:j+4] = np.array([ind1 + 2, ind1 + 3, ind1 + 2, ind1 + 3])

    # SE and SW quadrants together
    data[j+4:j+12] = K[2:, :].flat
    rows[j+4:j+12] = np.repeat(np.arange(ind1 + 2, ind1 + 4), 4)
    cols[j+4:j+12] = np.tile(np.arange(ind1, ind1 + 4), 2)

    j += 12

data[-4:] = 1.0
rows[-4] = 2 * num_nodes
rows[-3] = 2 * num_nodes + 1
rows[-2] = 0.0
rows[-1] = 1.0
cols[-4] = 0.0
cols[-3] = 1.0
cols[-2] = 2 * num_nodes
cols[-1] = 2 * num_nodes + 1

n_K = 2 * num_nodes + 2

In [50]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import splu
Ksp = coo_matrix((data, (rows, cols)), shape=(n_K, n_K)).tocsc()

In [59]:
Ksp.todense() # 24x24 (2(10+1)+2

matrix([[ 1.2e+04,  6.0e+02, -1.2e+04,  6.0e+02,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  1.0e+00,  0.0e+00],
        [ 6.0e+02,  4.0e+01, -6.0e+02,  2.0e+01,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  1.0e+00],
        [-1.2e+04, -6.0e+02,  2.4e+04,  0.0e+00, -1.2e+04,  6.0e+02,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00],
        [ 6.0e+02,  2.0e+01,  0.0e+00,  8.0e+01, -6.0e+02,  2.0e+01,
          0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00,  0.0e+00,  0

In [22]:
KLcomp = Component(compute_K_augmented, inputs=('K_local',), outputs=('K_aug',), 
                   indims=(num_elements*16,), outdims=(num_elements*16,))

array([[ 12000.,    600., -12000.,    600.],
       [   600.,     40.,   -600.,     20.],
       [-12000.,   -600.,  12000.,   -600.],
       [   600.,     20.,   -600.,     40.]])